In [ ]:
import numpy as np
np.seterr(all='ignore')

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 13})

from my_functions import *
from load_mocks import ensemble_mock

from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

In [ ]:
w_central = central_wavelength()
nb_fwhm_Arr = nb_fwhm(range(60))
w_lya = 1215.67
filter_tags = load_filter_tags()

In [ ]:
train_or_test = 'train'
survey_name = 'minijpas'
name_qso = 'QSO_100000_0'
name_qso_bad = f'QSO_double_{train_or_test}_{survey_name}_DR16_D_0'
name_gal = f'GAL_LC_{survey_name}_0'
name_sf = f'LAE_12.5deg_z2-4.25_{train_or_test}_{survey_name}_0'

pm_flx, pm_err, zspec, EW_lya, L_lya, is_qso, is_sf, is_gal, is_LAE, where_hiL, _ = ensemble_mock(
    name_qso, name_gal, name_sf, name_qso_bad, add_errs=True)


In [ ]:
gal_area = 5.54
bad_qso_area = 200
good_qso_area = 400

# the proportional factors are made in relation to bad_qso
# so bad_qso_factor = 1
gal_factor = bad_qso_area / gal_area
good_qso_factor = bad_qso_area / good_qso_area

In [ ]:
def gal_line_luminosities():
    lc = np.load('/home/alberto/almacen/David_lightcone/LightCone_513.npy')
    gal_z = lc['redshift']
    mask = (gal_z < 2)
    gal_line_flx = {
        'Hbeta': lc['FluxDust_Hbeta'][mask],
        'OIII': (lc['FluxDust_OIII5007'][mask] + lc['FluxDust_OIII4959'][mask]),
        'OII': (lc['FluxDust_OII3729'][mask] + lc['FluxDust_OII3727'][mask]),
    }

    dL = cosmo.luminosity_distance(gal_z[mask]).to(u.cm).value
    gal_line_L = {}
    for key in gal_line_flx.keys():
        gal_line_L[key] = np.log10(gal_line_flx[key] * 4*np.pi * dL ** 2)

    return gal_line_L
gal_line_L = gal_line_luminosities()

In [ ]:
mag = flux_to_mag(pm_flx[-2], w_central[-2])
mag[np.isnan(mag)] = 99.

N_sources = pm_flx.shape[1]
N_sources
print(f'qso {count_true(is_qso)}')
print(f'sf {count_true(is_sf)}')
print(f'gal {count_true(is_gal)}')

In [ ]:
PD_z_Arr = np.array([0.5, 1.5, 2.5, 3.5, 4.5, 5.5])
PD_counts_Arr = np.array([975471, 2247522, 1282573, 280401, 31368, 4322])

fig, ax = plt.subplots(figsize=(7, 6))

z_min = 0
z_max = 4.5
n_bins = 40
area_out = 400

bins = np.linspace(z_min, z_max, n_bins)
bins_c = [(bins[i] + bins[i + 1]) / 2 for i in range(len(bins) - 1)]
bin_size = (z_max - z_min) / (n_bins - 1)

z_hist, _ = np.histogram(zspec[is_qso], bins=bins)

ax.step(bins_c, z_hist / bin_size / area_out, label='z out')

area_in = 10_000
ax.plot(PD_z_Arr, PD_counts_Arr / 1 / area_in, marker='s', label='z in')

ax.set_yscale('log')
ax.set_ylabel('$\Delta z^{-1}\,\mathrm{deg}^{-2}$', fontsize=15)
ax.set_xlabel('z', fontsize=15)
ax.set_ylim((1, 1e3))
ax.legend(fontsize=15)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

bins = np.linspace(15, 25, 30)
bins_c = [bins[i : i + 2].sum() * 0.5 for i in range(len(bins) - 1)]
h, _ = np.histogram(mag[is_qso], bins=bins)
bin_size = (25 - 15) / (50 - 1)

ax.step(bins_c, h / bin_size / 400)

ax.set_xlabel('r (magAB)', fontsize=15)
ax.set_ylabel('$\Delta r^{-1}\,\mathrm{deg}^{-2}$', fontsize=15)
ax.set_yscale('log')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

ax.hist(L_lya[is_qso & (mag < 23)], bins=np.linspace(42, 45.5, 50), cumulative=True, density=True)
# ax.set_yscale('log')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

L_min = 42
L_max = 46
n_bins = 30
z_min = 2.2
z_max = 3.5
area = 400

bins = np.linspace(L_min, L_max, n_bins)
bins_c = [(bins[i] + bins[i + 1]) / 2 for i in range(len(bins) - 1)]
bin_size = (L_max - L_min) / (n_bins - 1)
volume = z_volume(z_min, z_max, area)

z_mask = (zspec > z_min) & (zspec < z_max)
L_hist, _ = np.histogram(L_lya[is_qso & z_mask], bins=bins)
L_hist_ewmin, _ = np.histogram(L_lya[is_qso & z_mask & (EW_lya > 30)], bins=bins)

ax.step(bins_c, L_hist / volume / bin_size, label='LF out')

Lx = np.logspace(L_min, L_max, 10000)
# Daniele's LF
# phistar1 = 3.33e-6
# Lstar1 = 44.65
# alpha1 = -1.35
# Zhang's LF
phistar1 = 10 ** -5.85
Lstar1 = 44.6
alpha1 = -1.2
Phi = schechter(Lx, phistar1, 10 ** Lstar1, alpha1) * Lx * np.log(10)

ax.plot(np.log10(Lx), Phi, label='LF in')

ax.set_yscale('log')
ax.set_ylabel(r'$\Phi$ [Mpc$^{-1}$ ($\Delta\log L$)$^{-1}$]', fontsize=15)
ax.set_xlabel(r'$\log L$', fontsize=15)
ax.set_ylim((1e-8, 3e-4))
ax.legend(fontsize=15)

plt.show()

In [ ]:
ew0_cut = 15
ew_other = 400

# Lya search
cont_est_lya, cont_err_lya = estimate_continuum(pm_flx, pm_err, IGM_T_correct=True)
line = is_there_line(pm_flx, pm_err, cont_est_lya, cont_err_lya, ew0_cut)
lya_lines, lya_cont_lines, line_widths = identify_lines(
    line, pm_flx, cont_est_lya, first=True, return_line_width=True
)
lya_lines = np.array(lya_lines)

# Other lines
cont_est_other, cont_err_other = estimate_continuum(pm_flx, pm_err, IGM_T_correct=False)
line_other = is_there_line(pm_flx, pm_err, cont_est_other, cont_err_other,
    ew_other, obs=True)
other_lines = identify_lines(line_other, cont_est_other, pm_err)

# Compute z
z_Arr = np.zeros(N_sources)
z_Arr[np.where(np.array(lya_lines) != -1)] =\
    z_NB(np.array(lya_cont_lines)[np.where(np.array(lya_lines) != -1)])

nice_z = np.abs(z_Arr - zspec) < 0.16

# %xdel cont_est_other
%xdel cont_err_other

In [ ]:
mag_min = 17
mag_max = 24

nb_min = 5
nb_max = 20
# nb_min = 16
# nb_max = 23

nbs_to_consider = np.arange(nb_min, nb_max + 1)

nb_cut = (np.array(lya_lines) >= nb_min) & (np.array(lya_lines) <= nb_max)

z_min = (w_central[nb_min] - nb_fwhm_Arr[nb_min] * 0.5) / w_lya - 1
z_max = (w_central[nb_max] + nb_fwhm_Arr[nb_max] * 0.5) / w_lya - 1
print(f'z interval: ({z_min:0.2f}, {z_max:0.2f})')

z_cut = (z_min < z_Arr) & (z_Arr < z_max)
zspec_cut = (z_min < zspec) & (zspec < z_max)
mag_cut = (mag > mag_min) & (mag < mag_max)

snr = np.empty(N_sources)
for src in range(N_sources):
    l = lya_lines[src]
    snr[src] = pm_flx[l, src] / pm_err[l, src]

nice_lya_mask = z_cut & mag_cut & (snr > 6)
nice_lya = nice_lya_select(
    lya_lines, other_lines, pm_flx, pm_err, cont_est_lya, z_Arr, mask=nice_lya_mask
)

In [ ]:
EW_nb_Arr, EW_nb_e, L_Arr, L_e_Arr, flambda, flambda_e = EW_L_NB(
    pm_flx, pm_err, cont_est_lya, cont_err_lya, z_Arr, lya_lines, N_nb=0
)

In [ ]:
count_true(is_sf)

In [ ]:
selected = np.random.permutation(
    np.where(
        nice_lya & is_qso & nice_z & (mag > 22)
    )[0]
)
print(count_true(selected))

# qso_lines = [1025.7220, 1397.61, 1549.48, 1908.73, 2799.12]
# Actually gal lines
# qso_lines = [4861, 5007, 3727, 6549, 6564, 6585]
# This is the peak of the gal contaminant distribution
qso_lines = [3200]
pm_flx[0, :] = 99999
pm_flx[-4, :] = 99999
pm_err[0, :] = 0
pm_err[-4, :] = 0

gal_src = np.where(is_gal)[0]
for i, src in enumerate(selected):
    if i == 10: break
    print(src)
    # print(f'L_lya = {L_lya[src]}')
    print(f'L_Arr = {L_Arr[src]}')
    print(f'EW = {EW_nb_Arr[src] * (1 + z_Arr[src])}')
    lya_obs_w = w_central[lya_lines[src]]
    other_lines_w = [w_central[i] for i in other_lines[src]]
    lya_real_w = w_lya * (1 + zspec[src])

    fig = plt.figure(figsize=(10, 5))
    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src], e17scale=True, set_ylim=False)

    ax.axvline(lya_obs_w, linestyle='--', color='r', label='Retrieved Lya line')
    ax.axvline(lya_real_w, color='g', label='Real Lya line')
    for w in other_lines_w:
        ax.axvline(w, linestyle='--', color='dimgray')
    for w in qso_lines:
        qso_obs_w = w * (1 + zspec[src])
        ax.axvline(qso_obs_w, linestyle=':', color='orange')
    print(other_lines[src])

    ax.plot(w_central[:56], cont_est_lya[:, src] * 1e17)
    # ax.plot(w_central[:56], cont_est_other[:, src], ls='--')

    # ax.set_title(f'zspec={zspec[src]:0.3f}, z_Arr={z_Arr[src]:0.3f}')
    ax.legend()

    ax.set_ylim((-1, 1))
    ax.set_xlim((3000, 10000))

    plt.show()

In [ ]:
def do_this():
    i = flux_to_mag(pm_flx[-1], w_central[-1])
    r = flux_to_mag(pm_flx[-2], w_central[-2])
    g = flux_to_mag(pm_flx[-3], w_central[-3])
    gr_err = (pm_err[-1] ** 2 + pm_err[-2] ** 2) ** 0.5

    fig, ax = plt.subplots(figsize=(7, 6))

    where = (nice_lya & is_qso & nice_z)
    bins = np.linspace(-1, 1, 11)
    to_hist = ((r - g) * (i - r))[where]
    ax.hist(to_hist, bins, histtype='step', density=True, linewidth=2)
    
    where = (nice_lya & is_sf & nice_z)
    to_hist = ((r - g) * (i - r))[where]
    ax.hist(to_hist, bins, histtype='step', density=True, linewidth=2)

    where = (nice_lya & is_gal)
    to_hist = ((r - g) * (i - r))[where]
    ax.hist(to_hist, bins, histtype='step', density=True, linewidth=2)

    ax.set_xlabel('(r - g) * (i - r)', fontsize=15)

    plt.show()

do_this()

In [ ]:
def contour_curve(xx, yy, bins_xx, bins_yy):
    Z, x, y = np.histogram2d(xx, yy, bins=(bins_xx, bins_yy))

    H_min = np.amin(Z)
    H_max = np.amax(Z)

    y_centers = 0.5 * ( y[1:] + y[:-1] )
    x_centers = 0.5 * ( x[1:] + x[:-1] )

    N_bins = 10000

    H_Arr = np.linspace(H_min , H_max , N_bins)[::-1]

    fact_up_Arr = np.zeros( N_bins )

    TOTAL_H = np.sum(Z)

    for iii in range(0, N_bins):

        mask = Z > H_Arr[iii]

        fact_up_Arr[iii] = np.sum(Z[ mask ]) / TOTAL_H

    H_value_68 = np.interp(0.683, fact_up_Arr, H_Arr) # 1sigma
    H_value_95 = np.interp(0.954, fact_up_Arr, H_Arr) # 2sigma
    H_value_90 = np.interp(0.900, fact_up_Arr, H_Arr) # 2sigma
    H_value_99 = np.interp(0.997, fact_up_Arr, H_Arr) # 2sigma
    # levels = [H_value_99]
    levels = [H_value_90, H_value_68]

    return x_centers, y_centers, Z.T, levels

In [ ]:
def do_this(c1, c2, xlabel, ylabel, xlims, ylims, curve=None):
    c1[~np.isfinite(c1)] = 99.
    c2[~np.isfinite(c2)] = 99.

    fig, ax = plt.subplots(figsize=(6, 6))

    bins_xx = np.linspace(-5.5, 3.5, 30)
    bins_yy = np.linspace(-2.5, 3, 30)

    where = (is_qso & zspec_cut & nice_lya)
    x = c1[where]
    y = c2[where]
    x_centers, y_centers, Z, levels = contour_curve(x, y, bins_xx, bins_yy)
    ax.contour(x_centers, y_centers, Z, levels=levels,
               colors='C0', linewidths=2, zorder=22)
    ax.contour(x_centers, y_centers, Z, levels=levels,
               colors='k', linestyles='-', linewidths=3, zorder=21)
    ax.scatter(x, y, alpha=0.3, s=5, zorder=1, rasterized=True)

    where = (is_sf & zspec_cut & nice_lya)
    x = c1[where]
    y = c2[where]
    x_centers, y_centers, Z, levels = contour_curve(x, y, bins_xx, bins_yy)
    ax.contour(x_centers, y_centers, Z, levels=levels,
               colors='C1', linewidths=2, zorder=12)
    ax.contour(x_centers, y_centers, Z, levels=levels,
               colors='k', linestyles='-', linewidths=3, zorder=11)
    ax.scatter(x, y, alpha=0.3, s=5, zorder=2, rasterized=True)

    where = (is_gal & nice_lya)
    x = c1[where]
    y = c2[where]
    x_centers, y_centers, Z, levels = contour_curve(x, y, bins_xx, bins_yy)
    ax.contour(x_centers, y_centers, Z, levels=levels,
               colors='C2', linewidths=2, zorder=32)
    ax.contour(x_centers, y_centers, Z, levels=levels,
               colors='k', linestyles='-', linewidths=3, zorder=31)
    ax.scatter(x, y, alpha=0.3, s=5, zorder=3, rasterized=True)

    if curve is not None:
        x = np.linspace(-10, 10, 1000)
        ax.plot(x, curve(x), color='red', linewidth=2,
                label='Color cut', zorder=99)

    # Dummy points for the legend
    ax.scatter(99, 99, c='C0', label='QSO')
    ax.scatter(99, 99, c='C1', label='SF LAE')
    ax.scatter(99, 99, c='C2', label='Galaxy')

    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    ax.set_xlabel(xlabel, fontsize=15)
    ax.set_ylabel(ylabel, fontsize=15)
    ax.legend(fontsize=12)

    filename = 'figures/ColorColor_contaminants.pdf'
    fig.savefig(filename, format='pdf', bbox_inches='tight',
                facecolor='w', edgecolor='w', backend='PDF')
    plt.show()


i = flux_to_mag(pm_flx[-1], w_central[-1])
r = flux_to_mag(pm_flx[-2], w_central[-2])
g = flux_to_mag(pm_flx[-3], w_central[-3])
u = flux_to_mag(pm_flx[-5], w_central[-5])

c1 = r - i
c2 = g - r
xlabel = 'r - i'
ylabel = 'g - r'
xlims = [-3, 2.5]
ylims = [-1.25, 3.2]
def curve(x): return -1.5 * x + 1.7


do_this(c1, c2, xlabel, ylabel, xlims, ylims, curve)

# c1 = i - u
# c2 = r - i
# xlabel = 'i - J1007'
# ylabel = 'r - i'
# xlims = [-3, 3]
# ylims = [-2, 2]
# def curve(x): return 0 * x

# do_this(c1, c2, xlabel, ylabel, xlims, ylims)


In [ ]:
def do_this(maskkk):
    fig, ax = plt.subplots(figsize=(8, 4))

    mask = nice_lya & is_gal & z_cut & maskkk
    w0 = w_central[lya_lines[mask]] / (1 + zspec[mask])

    bin_min = 1400
    bin_max = 5250
    bins = np.linspace(bin_min, bin_max, 70)
    bins_c = [bins[i: i + 2].sum() * 0.5 for i in range(len(bins) - 1)]

    hist, _ = np.histogram(w0, bins)
    ax.fill_between(bins_c, hist * gal_factor / 200, step='pre', color='dimgray')

    gal_line_w = [2799, 4861, 3727, 5008]
    gal_line_name = ['MgII', r'H$\beta$', 'OII', 'OIII', ]
    for w, name in zip(gal_line_w, gal_line_name):
        ax.axvline(w, color='orange', linestyle='--', zorder=-99)
        ax.text(w - 70, 130, name, fontsize=13)

    ax.set_xlim(bin_min, bin_max)
    ax.set_ylim(1e-2, 100)
    ax.set_yscale('log')
    ax.set_xlabel('$\lambda_0$ ($\AA$)', fontsize=14)
    ax.set_ylabel('Sources density (deg$^{-2})$', fontsize=14)

    fig.tight_layout()
    fig.savefig(f'figures/GAL_contaminants_w0_hist_ew0min{ew0_cut}.pdf', bbox_inches='tight',
                facecolor='w', edgecolor='w')
    plt.show()

    fig, ax = plt.subplots(figsize=(8, 4))

    bins = np.linspace(890, 3700, 70)
    bins_c = [bins[i: i + 2].sum() * 0.5 for i in range(len(bins) - 1)]

    mask = nice_lya & is_qso & (zspec < 2) & maskkk
    w0 = w_central[lya_lines[mask]] / (1 + zspec[mask])
    hbad, _ = np.histogram(w0, bins)
    mask = nice_lya & is_qso & (zspec > 2)
    w0 = w_central[lya_lines[mask]] / (1 + zspec[mask])
    hgood, _ = np.histogram(w0, bins)

    ax.fill_between(bins_c, (hbad + hgood * 0.5) / 200, step='pre', color='dimgray')

    qso_line_w = [1549.48, 1908.73, 2799.12, 2326.00, 1215.67, 1025]
    qso_line_name = ['CIV', 'CIII', 'MgII', 'CII', r'Ly$\alpha$', r'Ly$\beta$']

    for w, name in zip(qso_line_w, qso_line_name):
        ax.axvline(w, color='orange', linestyle='--', zorder=-99)
        ax.text(w - 70, 190, name, fontsize=13)

    ax.set_yscale('log')
    ax.set_xlabel('$\lambda_0$ ($\AA$)', fontsize=14)
    ax.set_ylabel('Sources density (deg$^{-2})$', fontsize=14)
    ax.set_ylim(1e-2, 150)

    fig.tight_layout()
    fig.savefig(f'figures/QSO_contaminants_w0_hist_ew0min{ew0_cut}.pdf', bbox_inches='tight',
                facecolor='w', edgecolor='w')
    plt.show()

do_this((mag < 23.5) & (mag > 17))

In [ ]:
def do_this():
    fig, ax = plt.subplots(figsize=(7, 6))

    xx = zspec[is_gal & nice_lya]
    yy = z_Arr[is_gal & nice_lya]
    bins2d = [np.linspace(0, 1.5, 30), np.linspace(2.4, 3.7, 30)]

    ax.hist2d(xx, yy, bins2d)
    
    ax.set_ylabel(r'$z_\mathrm{NB}$', fontsize=15)
    ax.set_xlabel(r'$z_\mathrm{real}$', fontsize=15)

    ax.set_title('Selected galaxies', fontsize=20)

    plt.show()

do_this()

In [ ]:
# Probability of a source being randomly selected is 0.003
print(count_true(is_gal & (lya_lines == 9)) /
      count_true(is_gal))

In [ ]:
EW_Arr = np.empty(L_Arr.shape)
for src in range(N_sources):
    l = lya_lines[src]
    EW_Arr[src] = (pm_flx[l, src] / cont_est_lya[l, src] - 1) * nb_fwhm_Arr[l]

In [ ]:
def do_this():
    fig, ax = plt.subplots(figsize=(9, 5))

    bins = np.linspace(1, 4, 20 + 1)
    bins_c = [bins[i: i + 2].sum() * 0.5 for i in range(len(bins) - 1)]
    print(bins.shape, len(bins_c))
    mag_max = 24
    

    log_EW0_Arr = np.log10(EW_Arr / (1 + z_Arr))

    mask = is_qso & nice_lya & nice_z & (mag < mag_max)
    hist, _ = np.histogram(log_EW0_Arr[mask], bins=bins)
    ax.step(bins_c, hist / good_qso_area,
            label='QSO LAE', color='C0', linewidth=2)

    mask = is_qso & nice_lya & ~nice_z & (mag < mag_max)
    hist, _ = np.histogram(log_EW0_Arr[mask], bins=bins)
    ax.step(bins_c, hist / bad_qso_area, label='QSO contaminant', color='C0',
            linewidth=1, linestyle='--')

    mask = is_sf & nice_lya & nice_z & (mag < mag_max)
    hist, _ = np.histogram(log_EW0_Arr[mask], bins=bins)
    ax.step(bins_c, hist / bad_qso_area, label='SF LAE', color='C1', linewidth=2)

    mask = is_gal & nice_lya & (mag < mag_max)
    hist, _ = np.histogram(log_EW0_Arr[mask], bins=bins)
    ax.step(bins_c, hist / gal_area, label='Galaxy',
            color='C2', linestyle='--', linewidth=1)

#     ax.axvline(np.log10(15), color='orange', linestyle='--')

    ax.set_xlabel(r'log Ly$\alpha$ EW$_0$', fontsize=15)
    ax.set_ylabel('deg$^{-2}$', fontsize=16)
    ax.legend(fontsize=16)

    ax.set_yscale('log')

#     ax.set_ylim(1, 3e3)
    ax.set_xlim(1, 4)

    N_sel = (
        count_true(nice_lya & is_sf)
        + count_true(nice_lya & is_qso & is_LAE) * good_qso_factor
        + count_true(nice_lya & is_qso & ~is_LAE)
        + count_true(nice_lya & is_gal) * gal_factor
    )
    to_save_hist = (
        np.histogram(log_EW0_Arr[nice_lya & is_sf & nice_z], bins)[0]
        + np.histogram(log_EW0_Arr[nice_lya & is_qso & nice_z], bins)[0] * good_qso_factor
    ) / N_sel
    np.save('npy/Expected_EW_hist.npy', to_save_hist)
    cont_hist = (
        np.histogram(log_EW0_Arr[nice_lya & is_qso & ~nice_z], bins)[0]
        + np.histogram(log_EW0_Arr[nice_lya & is_gal], bins)[0] * gal_factor
    ) / N_sel
    np.save('npy/Cont_EW_hist.npy', cont_hist)

do_this()
